# Percentual de viagens de pedestres em relação a população da área

Caderno Jupyter referente à **Issue #5**: *Mapa coroplético de origens e destinos de pedestres com filtros e granularidades* 

**APENAS MUNICÍPIO DE SÃO PAULO**

Comparação de viagens de pedestres e população da região (zona OD ou distrito)

---

In [31]:
import pandas as pd
import geopandas
import folium

### Carregando datasets e convertendo para CRS em comum 

Carregando datasets

In [32]:
od23_path = "../../data/OD/ODS/od23_all.csv" 
od = pd.read_csv(od23_path)

od_districts_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Distritos_2023_region.shp" 
od_districts = geopandas.read_file(od_districts_shapefile_path)

od_zones_shapefile_path = "../../data/OD/OD-2023/002_Site Metro Mapas_190225/Shape/Zonas_2023.shp" 
od_zones = geopandas.read_file(od_zones_shapefile_path)

Filtrando zonas OD que pertencem ao município de SP

In [33]:
od_zones = od_zones.query('NumeroMuni == 36')

Convertendo CRS

In [34]:
od_districts = od_districts.to_crs("EPSG:4326")
od_zones = od_zones.to_crs("EPSG:4326")

Cálculo da população por zonas OD

In [35]:
pop_sum_zone = (
    od.groupby("ZONA_O")["NO_MORAD"].sum()
    .reset_index()
    .rename(columns={"ZONA_O": "NumeroZona", "NO_MORAD": "TOTAL_POP"})
)

Soma do número do total de moradores no domicílio (coluna 14)

In [36]:
od["NO_MORAD"].sum()

np.int64(360947)

## Calcular viagens de pedestres / população da área

- Por zona (ZONA_O)
- Por distrito

### Viagens por zona OD / população

In [37]:
# 1) Filtrando apenas pedestres 
pedestrians_walk = od.query('MODOPRIN == 17')

# 2) Total de viagens para pedestres
zone_walk = (
    pedestrians_walk.groupby("ZONA_O")["FE_VIA"].sum()
    .reset_index()
    .rename(columns={"ZONA_O": "NumeroZona", "FE_VIA": "VIAGENS_PED"})
)

# 3) Merge do total de viagens / total de viagens de pedestres por zonas OD
zone_share = zone_walk.merge(
    pop_sum_zone,
    on="NumeroZona",
    how="inner"
)

# 4) Cálculo da representatividade de pedestres
zone_share["PED_POP"] = 100 * zone_share["VIAGENS_PED"] / zone_share["TOTAL_POP"]

# 5) Ordena decrescentemente por percentual de viagens de pedestres / população
zone_share = zone_share.sort_values("PED_POP", ascending=False)
zone_share.head()

,NumeroZona,VIAGENS_PED,TOTAL_POP,PED_POP
375,395,411.944319,1,41194.431942
249,258,33620.495741,283,11880.033831
448,473,81855.563681,720,11368.828289
172,178,7237.368242,69,10488.939481
447,472,36222.720897,350,10349.348828


### Viagens por distrito

In [38]:
# 1) Junta zonas com distritos para agregação futura de distritos
zone_district = od_zones[["NumeroZona", "NumDistrit", "NomeDistri"]].merge(
    zone_share, on="NumeroZona", how="left"
).fillna({"VIAGENS_PED": 0, "TOTAL_POP": 0, "PED_POP": 0})

# 2) Agregação de viagens de pedestres por distrito
district_walk = (
    zone_district.groupby(["NumDistrit", "NomeDistri"])["VIAGENS_PED"].sum()
    .reset_index()
)

# 3) Total de viagens de todos os meios por distrito
pop_sum_district = (
    zone_district.groupby("NumDistrit")["TOTAL_POP"].sum()
    .reset_index()
)

# 4) Merge do total de viagens / total de viagens de pedestres por distritos
district_share = district_walk.merge(
    pop_sum_district,
    on="NumDistrit",
    how="inner"
)

# 5) Cálculo da representatividade de pedestres
district_share["PED_POP"] = 100 * district_share["VIAGENS_PED"] / district_share["TOTAL_POP"]

# 6) ordena decrescentente de acordo com o percentual
district_share = district_share.sort_values("PED_POP", ascending=False)
district_share.head()

,NumDistrit,NomeDistri,VIAGENS_PED,TOTAL_POP,PED_POP
12,13,Cachoeirinha,90720.455654,1227.0,7393.680167
85,86,Vila Curuçá,103446.016134,1527.0,6774.460782
76,77,São Rafael,79496.340880,1193.0,6663.565874
77,78,Sapopemba,129992.562125,1976.0,6578.570958
29,30,Grajaú,167952.313309,2564.0,6550.402235


## Merge com geometrias

Juntar com geometrias

In [39]:
# zonas
zones_map = od_zones.merge(zone_share, on="NumeroZona", how="left").fillna(0)

# distritos
districts_map = od_districts.merge(district_share, left_on="NumeroDist", right_on="NumDistrit", how="left").fillna(0)

Filtrando distritos que pertençam São Paulo

In [40]:
districts_map = districts_map.query('PED_POP > 0')

## Mapa

Criar mapa Folium com LayerControl

In [41]:
# centro aproximado (SP)
m = folium.Map(location=[-23.55, -46.63], zoom_start=10)

Camada Zona OD

In [42]:
# camada por zona
folium.Choropleth(
    geo_data=zones_map.to_json(),
    name="Zonas OD",
    data=zones_map,
    columns=["NumeroZona", "PED_POP"],
    key_on="feature.properties.NumeroZona",
    fill_color="YlGnBu",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Viagens de pedestres (%) - Zona OD"
).add_to(m)

# adiciona tooltip + popup para zonas
folium.GeoJson(
    zones_map,
    style_function=lambda x: {"fillOpacity": 0, "color": "transparent"}, 
    tooltip=folium.GeoJsonTooltip(
        fields=["NomeZona", "PED_POP"],
        aliases=["Zona OD:", "Pedestre / População"],
        localize=True,
        sticky=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["NomeZona", "VIAGENS_PED", "TOTAL_POP", "PED_POP"],
        aliases=["Zona OD:", "Viagens:", "Total população", "Pedestre / População"],
        localize=True
    ),
    name="zonas od - pedestre / população",
    show=True
).add_to(m)

Camada Distrito

In [43]:
# camada por distrito
folium.Choropleth(
    geo_data=districts_map.to_json(),
    name="Distritos",
    data=districts_map,
    columns=["NumeroDist", "PED_POP"],
    key_on="feature.properties.NumeroDist",
    fill_color="OrRd",
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name="Viagens de pedestres (%) - Distritos",
    show=False
).add_to(m)

# adiciona tooltip + popup para distritos
folium.GeoJson(
    districts_map,
    style_function=lambda x: {"fillOpacity": 0, "color": "transparent"},
    tooltip=folium.GeoJsonTooltip(
        fields=["NomeDistri_x", "PED_POP"],
        aliases=["Distrito:", "Pedestre / População"],
        localize=True,
        sticky=True
    ),
    popup=folium.GeoJsonPopup(
        fields=["NomeDistri_x", "VIAGENS_PED", "TOTAL_POP", "PED_POP"],
        aliases=["Distrito:", "Viagens pedestres:", "Total população", "Pedestre / População"],
        localize=True
    ),
    name="distrito - pedestre / população",
    show=False
).add_to(m)

Salvar mapa

In [44]:
# adicionar controles
folium.LayerControl().add_to(m)

m.save("../built_maps/ped_trips_by_pop.html")